In [ ]:
# Задание
В качестве данных вам выдан датасет  

### Подробно про линейную регрессию можете прочитать тут: [пост Дъяконова](https://alexanderdyakonov.wordpress.com/2019/10/31/%D0%BB%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D0%B0%D1%8F-%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F/)

In [33]:
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns

## Препроцессинг

Ссылка на датасет: https://www.kaggle.com/datasets/altavish/boston-housing-dataset?resource=download (вам нужно скачать его, используя google почту), либо скачайте c [github-репозитория](https://github.com/liskiran/school3_lessons)

In [52]:
data = pd.read_csv("HousingData.csv")

In [54]:
cols = data.columns
cols

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [55]:
features_names = cols[:-1] # Название колонок признаков
target_name = cols[-1] # Название колонки целевой переменной

`x` - Признаки - (eng: Features) (наши данные, на чем будем обучать модель) \
`y` - Целевые переменные - (eng - Targets) (то, что хотим предсказать)

In [56]:
x, y = data[features_names], data[target_name]

In [57]:
x

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,NaN
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48


## Задание 1
1. Разделите датасет с помощью [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
---
### Разобьем наши данные на тренировочную и тестовую выборки:
![](https://algotrading101.com/learn/wp-content/uploads/2020/06/training-validation-test-data-set.png)

### Зачем мы это делаем:
Зеленым нарисована истинная зависимость.

Синим - выборка из точной зависимости + шум.

Красным - построенная по этим данным модель.

Модель хорошая? Почему?

![](https://i.stack.imgur.com/JezCW.png)


Мы хотим оценивать качество нашей модели. Т.е. уметь понимать, как проявит себя модель на данных, которые она не видела.

На тренировочных данных это делать плохо, потому что модель "видела" эти данные. Т.е. мы решали оптимизационную задачу и параметры модели подбирались таким образом, чтобы "угодить" на тренировочной выборке. Но оптимальность на тренировочной выборке совершенно не означает хорошее качество на новых данных.

Например, можно придумать такую модель машинного обучения: запомним все признаки каждого объекта 

```python
def get_model(x_train: np.ndarray, y_train: np.ndarray) -> dict:    
    model = {}
    for i in range(len(x_train)):
        model[tuple(x_train[i])] = y[i]
    return model
```

А предсказывать она будет так:

```python
def predict(model: dict, x: np.ndarray) -> float:
    return model[tuple(x)]
```

На тренировочной выборке качество у такой модели будет идеальным. Но на новых данных мы от этой модели ничего кроме ``` KeyError``` не получим

---

### Копипаста из [википедии](https://ru.wikipedia.org/wiki/Переобучение) про переобучение:

Переобучение (переподгонка, пере- в значении «слишком», англ. overfitting) в машинном обучении и статистике — явление, когда построенная модель хорошо объясняет примеры из обучающей выборки, но относительно плохо работает на примерах, не участвовавших в обучении (на примерах из тестовой выборки).

Это связано с тем, что при построении модели («в процессе обучения») в обучающей выборке обнаруживаются некоторые случайные закономерности, которые отсутствуют в генеральной совокупности.

Иными словами - модель запоминает огромное количество всех возможных примеров вместо того, чтобы научиться подмечать особенности.

Даже тогда, когда обученная модель не имеет чрезмерного количества параметров, можно ожидать, что эффективность её на новых данных будет ниже, чем на данных, использовавшихся для обучения[1]. В частности, значение коэффициента детерминации будет сокращаться по сравнению с исходными данными обучения.

Способы борьбы с переобучением зависят от метода моделирования и способа построения модели. Например, если строится дерево принятия решений, то можно обрезать некоторые его ветки в процессе построения.

---

![](https://static.packt-cdn.com/products/9781838556334/graphics/C12624_07_02.jpg)

---

2. Найдите пустые значение в таблице (`NaN`) и замените их на средние значения (в каждом столбце - свое среднее значение)
### Чем может быть полезна нормировка?
- Градиентные методы обычно лучше сходятся, если признаки из +- одинаковых диапазонов
- Можно начинать интерпретировать веса как важности признаков
3. Нормализуйте значения - [preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html), используя библеотику `sklearn` (такой метод используется для нормализации данных, модель будет лучше обучаться)

## Задание 2 
* Обучите модель [Линейной регрессии](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) и [Полиномиальной регрессии](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

## Задание 3
- Выведите метрики после обучения моделей, такие как [MSE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html), [RMSE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.root_mean_squared_error.html), [MAE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html)
- Сделайте выводы какая модель лучше и почему